In [36]:
import os
import re

import pandas as pd
os.getcwd()

'C:\\Users\\ge47yej\\Siby_Research\\Mg-ML+GAS_from_FELIX\\to_upload_Github'

In [37]:
rawstruc_fn = "./data/siby-manganese-raw"
datafolders = []
for folder, dirnames, filenames in os.walk(rawstruc_fn):
    if "CONTCAR" in filenames:
        datafolders.append(folder)
print(len(datafolders))

119


In [38]:
datafolders2canonical = {}
site_lookup = {"OT-h" : "Oh", "OT-v" : "Ov", "OT-v1" : "Ov1",
               "Br-h" : "Bh", "Br-v" : "Bv", "Br-v1" : "Bv1", 
               "H-h" : "Hh", "H-v" : "Hv", "H-v1" : "Hv1", 
               "defect-v1" : "Dv1", 'defcet-v' : "Dv", 'defect-v': "Dv",
               'defect-h' : "Dh", "def" : "D", "def2" : "D2"}
               
for df in datafolders:
    naming = re.match(fr"{rawstruc_fn}[/\\][0-9]-(.*?)[/\\][0-9]-(.*?)[/\\][0-9]+-(.*)",
                     df)
    if not naming:
        print("PROBLEM MATCHING: ", df)
        continue
    adsorbant, sys, site = naming.groups()
    try:
        canonical_site = site_lookup[site]
    except Exception as e:
        print(df)
        raise e
    
    try:
        with open(os.path.join(df, "OUTCAR")) as outcar:
            final_energy = float(
                re.findall(r'free energy\s+TOTEN\s+=\s+([0-9-]+.[0-9]+) eV', outcar.read())[-1])
    except FileNotFoundError as e:
        final_energy = None
        print("Missing OUTCAR", df)
    except Exception:
        final_energy = None
        print(df)
        
    datafolders2canonical[df] = (f"Mg-{sys}-{canonical_site}-{adsorbant}", final_energy)
    
keys2folders = {}
for k, v in datafolders2canonical.items():
    if v[0] not in keys2folders.keys():
        keys2folders[v[0]] = {"path" : k, "check_energy" : v[1]}
    else:
        print(v, ": ", k, all_files[v])
        break

Missing OUTCAR ./data/siby-manganese-raw\1-CO\1-PR\2-OT-h


In [39]:
data_df

,path,check_energy
Mg-PR-Hh-CO,./data/siby-manganese-raw\1-CO\1-PR\1-H-h,-38.994429
Mg-PR-Oh-CO,./data/siby-manganese-raw\1-CO\1-PR\2-OT-h,NaN
Mg-PR-Bh-CO,./data/siby-manganese-raw\1-CO\1-PR\3-Br-h,-38.946718
Mg-PR-Hv-CO,./data/siby-manganese-raw\1-CO\1-PR\4-H-v,-38.996568
Mg-PR-Ov-CO,./data/siby-manganese-raw\1-CO\1-PR\5-OT-v,-38.688407
...,...,...
Mg-DV2-D-CH4,./data/siby-manganese-raw\3-CH4\4-DV2\4-def,-44.088889
Mg-DV2-Hv-CH4,./data/siby-manganese-raw\3-CH4\4-DV2\5-H-v,-43.926195
Mg-DV2-Ov-CH4,./data/siby-manganese-raw\3-CH4\4-DV2\6-OT-v,-43.913053
Mg-DV2-Bv-CH4,./data/siby-manganese-raw\3-CH4\4-DV2\7-Br-v,-43.927009


In [40]:
# find the energy and compare it to the table by siby. then save!
data_df = pd.DataFrame.from_dict(keys2folders, orient="index")
labels_df = pd.read_excel("./data/Mg_data_for_ML.fixed.xlsx")
labels_df["System"] = labels_df["System"].apply(lambda s: s.replace("Mg-BV1", "Mg-DV1"))
labels_df["System"] = labels_df["System"].apply(lambda s: s.replace("Mg-BV2", "Mg-DV2"))

system_df = labels_df.merge(data_df, left_on="System", right_index=True)

mismatch = system_df[
    system_df["TE"] - system_df["check_energy"] > 0.0001][
    ["System", "TE", "check_energy", "path"]]

In [41]:
mismatch

,System,TE,check_energy,path


In [42]:
system_df.to_excel("data/dataset_pp.2022-12-09.xlsx")